In [47]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [48]:
file_path = "./batch_FPlzDKOMtP0tqtItZjhxN9PX_output.jsonl"
gt_data = pd.read_json(f"./nested_squares/config.json")

In [49]:
# Dictionary to map custom_id to message content
custom_id_to_content = {}

# Open the file and read line by line
with open(file_path, "r") as file:
    for line in file:
        json_obj = json.loads(line)
        # Extract custom_id and message content
        custom_id = json_obj.get("custom_id")
        message_content = json_obj["response"]["body"]["choices"][0]["message"][
            "content"
        ]
        # Map custom_id to message content
        if custom_id:
            custom_id_to_content[custom_id.replace("uid__", "")] = message_content

# Print the dictionary to verify
print(len(custom_id_to_content))

120


In [50]:
custom_id_to_content

{'nested_squares_depth_2_image_1_thickness_2.png': 'The image contains two squares: \n- One smaller square inside.\n- One larger square outside.\n\nSo, the total number of squares in the image is 2.',
 'nested_squares_depth_2_image_1_thickness_3.png': 'There are a total of 2 squares in the image. One smaller square is inside a larger square.',
 'nested_squares_depth_2_image_1_thickness_4.png': 'There are 2 squares in the image.',
 'nested_squares_depth_2_image_2_thickness_2.png': 'The image contains two squares: one small square inside a larger square. Therefore, there are a total of 2 squares in the image.',
 'nested_squares_depth_2_image_2_thickness_3.png': 'The image contains a total of 2 squares: one larger square and one smaller square inside it.',
 'nested_squares_depth_2_image_2_thickness_4.png': 'The image contains a total of three squares:\n\n1. One small, innermost square.\n2. One larger square surrounding the smaller square.\n3. One overall square that includes both the inne

In [51]:
# gt_data = (
#     gt_data.T
# )  # use indexx as a new column in the dataframe and called it image_path
# gt_data = gt_data.reset_index()
# gt_data = gt_data.rename(columns={"index": "image_path"})
# gt_data

In [52]:
# drop squares column
gt_data = gt_data.drop(columns=["squares"])

In [54]:
# ad a new column to gt_data with content of  custom_id_to_content (match key with image_path)
gt_data["message"] = gt_data["image_name"].map(custom_id_to_content)
gt_data

,image_name,depth,center,initial_size,reduction_factor,line_thickness,padding,message
0,nested_squares_depth_2_image_1_thickness_2.png,2,"[3.687501695943579, -1.6639490542139401]",10.592918,0.75,2,0.75,The image contains two squares: \n- One smalle...
1,nested_squares_depth_2_image_1_thickness_3.png,2,"[3.687501695943579, -1.6639490542139401]",10.592918,0.75,3,0.75,There are a total of 2 squares in the image. O...
2,nested_squares_depth_2_image_1_thickness_4.png,2,"[3.687501695943579, -1.6639490542139401]",10.592918,0.75,4,0.75,There are 2 squares in the image.
3,nested_squares_depth_2_image_2_thickness_2.png,2,"[4.438505912414158, -3.031304472680408]",8.928089,0.75,2,0.75,The image contains two squares: one small squa...
4,nested_squares_depth_2_image_2_thickness_3.png,2,"[4.438505912414158, -3.031304472680408]",8.928089,0.75,3,0.75,The image contains a total of 2 squares: one l...
...,...,...,...,...,...,...,...,...
115,nested_squares_depth_5_image_9_thickness_3.png,5,"[-0.9664633355834571, 1.7004872368012192]",11.484833,0.75,3,0.75,The image contains a total of 5 squares.
116,nested_squares_depth_5_image_9_thickness_4.png,5,"[-0.9664633355834571, 1.7004872368012192]",11.484833,0.75,4,0.75,The image contains 6 squares.
117,nested_squares_depth_5_image_10_thickness_2.png,5,"[-3.710606733934443, 2.55403635196378]",8.514063,0.75,2,0.75,The image contains 5 squares. Each square is n...
118,nested_squares_depth_5_image_10_thickness_3.png,5,"[-3.710606733934443, 2.55403635196378]",8.514063,0.75,3,0.75,The image contains a total of 5 squares.


In [55]:
import re


# Function to extract numbers from text, including zero represented as text
def extract_number(text):
    # Check for phrases that imply zero intersections
    if (
        "no intersections" in text
        or "zero line intersections" in text
        or "no line" in text
    ):
        return 0
    # First try to find digits
    match = re.search(r"\b\d+\b", text)
    if match:
        return int(match.group())
    # If no digits, look for text numbers (one, two, three, etc.)
    text_to_num = {"one": 1, "two": 2, "three": 3, "four": 4, "five": 5}
    for word, num in text_to_num.items():
        if word in text:
            return num
    return None


# Apply the function to the 'message' column
gt_data["extracted_number"] = gt_data["message"].apply(extract_number)
gt_data[["image_name", "message", "extracted_number"]]

,image_name,message,extracted_number
0,nested_squares_depth_2_image_1_thickness_2.png,The image contains two squares: \n- One smalle...,2
1,nested_squares_depth_2_image_1_thickness_3.png,There are a total of 2 squares in the image. O...,2
2,nested_squares_depth_2_image_1_thickness_4.png,There are 2 squares in the image.,2
3,nested_squares_depth_2_image_2_thickness_2.png,The image contains two squares: one small squa...,2
4,nested_squares_depth_2_image_2_thickness_3.png,The image contains a total of 2 squares: one l...,2
...,...,...,...
115,nested_squares_depth_5_image_9_thickness_3.png,The image contains a total of 5 squares.,5
116,nested_squares_depth_5_image_9_thickness_4.png,The image contains 6 squares.,6
117,nested_squares_depth_5_image_10_thickness_2.png,The image contains 5 squares. Each square is n...,5
118,nested_squares_depth_5_image_10_thickness_3.png,The image contains a total of 5 squares.,5


In [56]:
gt_data

,image_name,depth,center,initial_size,reduction_factor,line_thickness,padding,message,extracted_number
0,nested_squares_depth_2_image_1_thickness_2.png,2,"[3.687501695943579, -1.6639490542139401]",10.592918,0.75,2,0.75,The image contains two squares: \n- One smalle...,2
1,nested_squares_depth_2_image_1_thickness_3.png,2,"[3.687501695943579, -1.6639490542139401]",10.592918,0.75,3,0.75,There are a total of 2 squares in the image. O...,2
2,nested_squares_depth_2_image_1_thickness_4.png,2,"[3.687501695943579, -1.6639490542139401]",10.592918,0.75,4,0.75,There are 2 squares in the image.,2
3,nested_squares_depth_2_image_2_thickness_2.png,2,"[4.438505912414158, -3.031304472680408]",8.928089,0.75,2,0.75,The image contains two squares: one small squa...,2
4,nested_squares_depth_2_image_2_thickness_3.png,2,"[4.438505912414158, -3.031304472680408]",8.928089,0.75,3,0.75,The image contains a total of 2 squares: one l...,2
...,...,...,...,...,...,...,...,...,...
115,nested_squares_depth_5_image_9_thickness_3.png,5,"[-0.9664633355834571, 1.7004872368012192]",11.484833,0.75,3,0.75,The image contains a total of 5 squares.,5
116,nested_squares_depth_5_image_9_thickness_4.png,5,"[-0.9664633355834571, 1.7004872368012192]",11.484833,0.75,4,0.75,The image contains 6 squares.,6
117,nested_squares_depth_5_image_10_thickness_2.png,5,"[-3.710606733934443, 2.55403635196378]",8.514063,0.75,2,0.75,The image contains 5 squares. Each square is n...,5
118,nested_squares_depth_5_image_10_thickness_3.png,5,"[-3.710606733934443, 2.55403635196378]",8.514063,0.75,3,0.75,The image contains a total of 5 squares.,5


In [57]:
# count none or na in gt_data extracted_number
gt_data["extracted_number"].isna().sum()
# show their value in the message (same row)
gt_data[gt_data["extracted_number"].isna()]["message"]

Series([], Name: message, dtype: object)

In [58]:
# Cast depth to integer
gt_data["depth"] = gt_data["depth"].astype(int)

# Cast extracted_number to integer (it should already be integer from the extraction function, but just to ensure consistency)
gt_data["extracted_number"] = gt_data["extracted_number"].astype(int)

# Compare depth and extracted_number to check if they are the same
gt_data["is_correct"] = gt_data["depth"] == gt_data["extracted_number"]

# Display the DataFrame with the new column
gt_data[["image_name", "depth", "extracted_number", "is_correct"]]

,image_name,depth,extracted_number,is_correct
0,nested_squares_depth_2_image_1_thickness_2.png,2,2,True
1,nested_squares_depth_2_image_1_thickness_3.png,2,2,True
2,nested_squares_depth_2_image_1_thickness_4.png,2,2,True
3,nested_squares_depth_2_image_2_thickness_2.png,2,2,True
4,nested_squares_depth_2_image_2_thickness_3.png,2,2,True
...,...,...,...,...
115,nested_squares_depth_5_image_9_thickness_3.png,5,5,True
116,nested_squares_depth_5_image_9_thickness_4.png,5,6,False
117,nested_squares_depth_5_image_10_thickness_2.png,5,5,True
118,nested_squares_depth_5_image_10_thickness_3.png,5,5,True


In [59]:
gt_data["is_correct"].value_counts()

is_correct
False    64
True     56
Name: count, dtype: int64

In [60]:
100 * np.mean(gt_data["is_correct"])

46.666666666666664

In [67]:
# Assuming 'cleaned_data' DataFrame has columns 'line_thickness', 'num_intersections', and 'extracted_number'

# Convert line_thickness to an appropriate numeric type if necessary
gt_data["line_thickness"] = gt_data["line_thickness"].astype(int)

# Calculate accuracy for each thickness
accuracy_by_thickness = gt_data.groupby("line_thickness").apply(
    lambda df: (df["is_correct"]).mean()
)

accuracy_by_thickness = 100 * accuracy_by_thickness.round(2)
accuracy_by_thickness

/var/folders/y1/b5wjnkp96_b7cvzncx0zsry40000gn/T/ipykernel_87367/388290072.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  accuracy_by_thickness = gt_data.groupby("line_thickness").apply(


line_thickness
2    52.0
3    55.0
4    32.0
dtype: float64

In [66]:
# Convert line_thickness to an appropriate numeric type if necessary
gt_data["depth"] = gt_data["depth"].astype(int)

# Calculate accuracy for each thickness
accuracy_by_thickness = gt_data.groupby("depth").apply(
    lambda df: (df["is_correct"]).mean()
)

accuracy_by_thickness = 100 * accuracy_by_thickness.round(2)
accuracy_by_thickness

/var/folders/y1/b5wjnkp96_b7cvzncx0zsry40000gn/T/ipykernel_87367/210657226.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  accuracy_by_thickness = gt_data.groupby("depth").apply(


depth
2    77.0
3    23.0
4    20.0
5    67.0
dtype: float64